<b><font color="red">* To access the model, please load the models from .pth files in the "model_usage.ipynb". <br>** With some initial weight, certain models may not converge.</font></b>

In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn import metrics
import bentoml
import json
import torch.nn as nn
import torch.optim as optim
import time
import torch.nn.init as init
from torch.utils.data import TensorDataset, DataLoader

## Load, Preprocess, & Split data

In [2]:
df = pd.read_csv("./Data/features_3_sec.csv")
df

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,rock.00099.5.wav,66149,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock
9986,rock.00099.6.wav,66149,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock
9987,rock.00099.7.wav,66149,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock
9988,rock.00099.8.wav,66149,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock


In [3]:
def extract_name(x):
    return x.split('.')[0]+'.'+x.split('.')[1]

def number(x):
    return int(x.split('.')[1])

def label(x):
    d = {
        "blues": 0,
        "classical":1,
        "country": 2,
        "disco":3,
        "hiphop":4,
        "jazz": 5,
        "metal":6,
        "pop":7,
        "reggae":8,
        "rock":9
    }
    return d[x]

In [4]:
song = pd.DataFrame()
song['song'] = df['filename'].apply(extract_name)
song['label'] = df['label'].apply(label)
song['number'] = song['song'].apply(number)
df['number'] = df['filename'].apply(number)
df.drop(columns=['length', 'tempo', 'perceptr_mean','perceptr_var','label','filename', "rolloff_var", "spectral_bandwidth_mean", "rolloff_mean", "harmony_var", "zero_crossing_rate_mean", "zero_crossing_rate_var","mfcc1_mean","mfcc1_var","mfcc2_mean","mfcc2_var"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9990 entries, 0 to 9989
Data columns (total 45 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   chroma_stft_mean        9990 non-null   float64
 1   chroma_stft_var         9990 non-null   float64
 2   rms_mean                9990 non-null   float64
 3   rms_var                 9990 non-null   float64
 4   spectral_centroid_mean  9990 non-null   float64
 5   spectral_centroid_var   9990 non-null   float64
 6   spectral_bandwidth_var  9990 non-null   float64
 7   harmony_mean            9990 non-null   float64
 8   mfcc3_mean              9990 non-null   float64
 9   mfcc3_var               9990 non-null   float64
 10  mfcc4_mean              9990 non-null   float64
 11  mfcc4_var               9990 non-null   float64
 12  mfcc5_mean              9990 non-null   float64
 13  mfcc5_var               9990 non-null   float64
 14  mfcc6_mean              9990 non-null   

In [5]:
song

,song,label,number
0,blues.00000,0,0
1,blues.00000,0,0
2,blues.00000,0,0
3,blues.00000,0,0
4,blues.00000,0,0
...,...,...,...
9985,rock.00099,9,99
9986,rock.00099,9,99
9987,rock.00099,9,99
9988,rock.00099,9,99


In [6]:
x_train = df[df['number'] >= 20]
x_val = df[df['number'] < 20]
song_train = song[song['number'] >= 20]
song_val = song[song['number'] < 20]
x_val

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_var,harmony_mean,mfcc3_mean,mfcc3_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,number
0,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,117335.771563,-0.000078,-23.443724,364.081726,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,0
1,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,65671.875673,-0.000099,-20.718019,231.979767,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,0
2,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,75124.921716,-0.000066,-14.811666,192.448074,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,0
3,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,82913.639269,-0.000014,-18.758335,109.357529,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,0
4,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,60204.020268,0.000041,-19.961748,171.878754,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9187,0.510478,0.067812,0.137442,0.000471,2382.837801,95890.852456,47301.659445,0.000010,-37.752861,165.973831,...,18.839457,-6.600465,44.321831,5.637548,17.616732,-5.278373,20.246199,-0.877959,15.256322,19
9188,0.517307,0.063758,0.104017,0.002828,2252.497236,94959.815941,41463.920038,-0.000101,-43.459805,241.594330,...,26.437086,-8.344719,15.496710,2.172762,64.985870,-1.632867,14.844873,2.336645,23.016424,19
9189,0.422443,0.082976,0.126436,0.001261,2239.760106,262924.983760,82567.079338,-0.000063,-34.651443,269.977417,...,17.965687,-7.647231,29.996981,2.986220,26.933788,-3.504234,16.753084,0.204434,23.066519,19
9190,0.436105,0.079302,0.129293,0.000675,2066.207203,186924.659732,74889.709411,-0.000029,-32.433960,167.319763,...,19.505196,-6.679996,30.482504,2.490437,23.382385,-4.498179,22.776510,-1.911705,16.353846,19


In [7]:
x_train

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_var,harmony_mean,mfcc3_mean,mfcc3_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,number
200,0.274168,0.095494,0.082787,0.001400,1430.583940,90388.046462,67355.973409,0.000033,-7.124894,820.651794,...,75.780464,-8.392806,71.304054,-2.363670,83.485107,-1.457948,146.627640,0.821828,117.664154,20
201,0.285800,0.093292,0.064784,0.001630,1333.040315,84946.113534,97767.675787,-0.000048,7.262296,364.484314,...,37.606544,-6.054728,42.716873,-4.472951,67.612930,2.411800,41.252251,8.174487,29.740971,20
202,0.291340,0.095959,0.063447,0.000599,1274.004446,51841.907876,77624.137099,-0.000026,5.903938,455.457214,...,28.155912,-5.323036,36.842255,-1.868089,45.987854,-1.691187,57.396564,2.435588,48.908344,20
203,0.335422,0.096567,0.063272,0.001353,1492.238144,101402.408007,79519.599563,0.000012,3.326139,595.447144,...,67.726364,-4.983536,40.922844,-2.800988,30.736599,-5.176728,28.020010,0.217755,20.596766,20
204,0.307678,0.097503,0.054217,0.000619,1401.144165,58983.959478,67266.507714,-0.000014,13.077976,162.458420,...,49.347980,-3.173147,34.055576,0.417186,41.342144,-0.406688,72.350197,6.398019,72.829262,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,85931.574523,-0.000034,-31.418577,183.526886,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,99
9986,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,99727.037054,-0.000027,-36.626003,140.996994,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,99
9987,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,138762.841945,0.000015,-11.647097,1331.622681,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,99
9988,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,22860.992562,0.000004,-36.225224,187.097839,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,99


In [8]:
song_train

,song,label,number
200,blues.00020,0,20
201,blues.00020,0,20
202,blues.00020,0,20
203,blues.00020,0,20
204,blues.00020,0,20
...,...,...,...
9985,rock.00099,9,99
9986,rock.00099,9,99
9987,rock.00099,9,99
9988,rock.00099,9,99


In [9]:
song_val

,song,label,number
0,blues.00000,0,0
1,blues.00000,0,0
2,blues.00000,0,0
3,blues.00000,0,0
4,blues.00000,0,0
...,...,...,...
9187,rock.00019,9,19
9188,rock.00019,9,19
9189,rock.00019,9,19
9190,rock.00019,9,19


In [10]:
x_train= x_train.drop(columns=['number'])
x_val= x_val.drop(columns=['number'])
song_train= song_train.drop(columns=['number'])
song_val= song_val.drop(columns=['number'])

In [11]:
y_train = torch.tensor(torch.nn.functional.one_hot(torch.tensor(list(song_train["label"])), num_classes=10))
y_train
print(y_train.shape)

torch.Size([7994, 10])


C:\Users\User\AppData\Local\Temp\ipykernel_12920\564108120.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(torch.nn.functional.one_hot(torch.tensor(list(song_train["label"])), num_classes=10))


In [12]:
y_val = torch.tensor(torch.nn.functional.one_hot(torch.tensor(list(song_val["label"])), num_classes=10))
y_val

C:\Users\User\AppData\Local\Temp\ipykernel_12920\1782615230.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(torch.nn.functional.one_hot(torch.tensor(list(song_val["label"])), num_classes=10))


tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1]])

In [13]:
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val.values, dtype=torch.float32)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_train_tensor=x_train_tensor.to(device)
x_val_tensor=x_val_tensor.to(device)
y_train=y_train.to(device)
y_val=y_val.to(device)
y_train =y_train.float()
y_val =y_val.float()

# Model training

## Useful function

In [15]:
def cal_song_acc(song, pred):
    copy_song = song.reset_index()
    grouped = copy_song.groupby('song')
    true_labels = list(grouped["label"].mean().astype(int))
    # print(true_labels)
    aggregated_list = []
    max_locs_list = []
    for group_name, group_indices in grouped.groups.items():
        group_values = copy_song.loc[group_indices, 'label']
        group_list_indices = list(group_values.index)
        group_list = [pred[i] for i in group_list_indices]
        # Calculate sum or average for each column of the 2D list
        group_sum = np.sum(group_list, axis=0)  # or np.mean(group_list, axis=0) for average
        aggregated_list.append(group_sum)
        # Find the location of the maximum value in the aggregated array
        max_loc = np.argmax(group_sum)
        max_locs_list.append(max_loc)
    # print(max_locs_list)
    # print(metrics.accuracy_score(true_labels, max_locs_list))
    return metrics.accuracy_score(true_labels, max_locs_list)

def cal_acc(song, pred):
    max_locs_list = []
    for row in pred:
        max_index = torch.argmax(row).item()
        max_locs_list.append(max_index)
    return metrics.accuracy_score(list(song["label"]), max_locs_list)

def save_model(model, model_info, model_name, train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration):
    bentoml.pytorch.save_model(model_name, model)
    print("saved")
    logs = dict()
    logs["params"] = model_info
    logs["train_loss"] = train_loss
    logs["val_loss"] = val_loss
    logs["train_acc"] = train_acc
    logs["train_song_acc"] = train_song_acc
    logs["val_acc"] = val_acc
    logs["val_song_acc"] = val_song_acc
    logs["duration"] = duration
    latest_file_path = f"../../../bentoml/models/{model_name}/latest"
    with open(latest_file_path, "r") as file:
        tag = file.read().strip()
    file_path = f"json_log\{model_name}_{tag}"
    # Save dictionary as JSON file
    with open(file_path, 'w') as json_file:
        json.dump(logs, json_file)
    print("logged")

def train(num_epochs, model, optimizer, criterion, batch_size):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TensorDataset(x_train_tensor, y_train)
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        for x, y in train_loader:
            x=x.to(device)
            y=y.to(device)
            y=y.float()
            # print(x.device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
        model.eval()
        with torch.no_grad():
            train_loss = 0.0
            val_loss = 0.0
            
            outputs = model(x_train_tensor)
            train_loss += criterion(outputs, y_train).item()
            train_song_acc = cal_song_acc(song_train, outputs.to("cpu"))
            train_acc = cal_acc(song_train, outputs.to("cpu"))
            
            outputs = model(x_val_tensor)
            val_loss += criterion(outputs, y_val).item()
            val_song_acc = cal_song_acc(song_val, outputs.to("cpu"))
            val_acc = cal_acc(song_val, outputs.to("cpu"))
        print(f"Epoch {epoch + 1}:")
        print(f"  Train Loss: {train_loss}")
        print(f"  Train Song Accuracy: {train_song_acc}")
        print(f"  Train Accuracy: {train_acc}")
        print(f"  Validation Loss: {val_loss}")
        print(f"  Validation Song Accuracy: {val_song_acc}")
        print(f"  Validation Accuracy: {val_acc}")
    end_time = time.time()
    duration = end_time - start_time
    return train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration

## ANN

### Run 1
- Removing some highly correlated features to reduce the chance of overfitting and potentially aid in avoiding saddle points.
- Experimenting with a similar architecture to the one used with all features.

In [56]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(44, 88)
        self.fc2 = nn.Linear(88, 88)
        self.fc3 = nn.Linear(88, 44)
        self.fc4 = nn.Linear(44, 10)  # Adjust output size based on the number of classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

In [17]:
model_info = {
    "name": "ANN",
    "arch": "44-88-88-88-88-88-88-44-10",
    "criterion": "BCELoss",
    "lr":0.001,
    "epoch": 300,
    "batch_size":128,
    "optimizer":"Adam",
    "activation":"Sigmoid",
    "features": "Selected"
}

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 300
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 0.45225921273231506
  Train Song Accuracy: 0.16875
  Train Accuracy: 0.14811108331248438
  Validation Loss: 0.4588160216808319
  Validation Song Accuracy: 0.13
  Validation Accuracy: 0.1282565130260521
Epoch 2:
  Train Loss: 0.392641544342041
  Train Song Accuracy: 0.205
  Train Accuracy: 0.19239429572179134
  Validation Loss: 0.40350598096847534
  Validation Song Accuracy: 0.185
  Validation Accuracy: 0.16082164328657314
Epoch 3:
  Train Loss: 0.3494407534599304
  Train Song Accuracy: 0.21875
  Train Accuracy: 0.2070302727045284
  Validation Loss: 0.35406729578971863
  Validation Song Accuracy: 0.255
  Validation Accuracy: 0.20691382765531063
Epoch 4:
  Train Loss: 0.30991286039352417
  Train Song Accuracy: 0.31375
  Train Accuracy: 0.2656992744558419
  Validation Loss: 0.3186391294002533
  Validation Song Accuracy: 0.33
  Validation Accuracy: 0.24298597194388777
Epoch 5:
  Train Loss: 0.33569037914276123
  Train Song Accuracy: 0.2575
  Train Accuracy: 0.2271703

In [19]:
save_model(model, model_info, "ANN-selected", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)

Converting 'ANN-selected' to lowercase: 'ann-selected'.


saved
logged


## Run 2

In [20]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(44, 88)
        self.fc2 = nn.Linear(88, 88)
        self.fc3 = nn.Linear(88, 176)
        self.fc4 = nn.Linear(176, 176)
        self.fc5 = nn.Linear(176, 88)
        self.fc6 = nn.Linear(88, 44)
        self.fc7 = nn.Linear(44, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = torch.sigmoid(self.fc7(x))
        return x

In [21]:
model_info = {
    "name": "ANN",
    "arch": "44-88-88-88-176-176-88-44-10",
    "criterion": "BCELoss",
    "lr":0.0009,
    "epoch": 300,
    "batch_size":128,
    "optimizer":"Adam",
    "activation":"Sigmoid",
    "features": "Selected"
}

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.0009)
num_epochs = 300
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 3.1349570751190186
  Train Song Accuracy: 0.115
  Train Accuracy: 0.11746309732299225
  Validation Loss: 2.6910359859466553
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10170340681362726
Epoch 2:
  Train Loss: 0.37137266993522644
  Train Song Accuracy: 0.2425
  Train Accuracy: 0.20903177383037277
  Validation Loss: 0.3757443428039551
  Validation Song Accuracy: 0.22
  Validation Accuracy: 0.19288577154308617
Epoch 3:
  Train Loss: 0.31306561827659607
  Train Song Accuracy: 0.33375
  Train Accuracy: 0.2723292469352014
  Validation Loss: 0.3182772994041443
  Validation Song Accuracy: 0.285
  Validation Accuracy: 0.24248496993987975
Epoch 4:
  Train Loss: 0.30504509806632996
  Train Song Accuracy: 0.31125
  Train Accuracy: 0.2684513385038779
  Validation Loss: 0.3141351640224457
  Validation Song Accuracy: 0.305
  Validation Accuracy: 0.251503006012024
Epoch 5:
  Train Loss: 0.2929665148258209
  Train Song Accuracy: 0.32125
  Train Accuracy: 0.2748311233

In [23]:
save_model(model, model_info, "ANN-selected", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)

Converting 'ANN-selected' to lowercase: 'ann-selected'.


saved
logged


## Run 3

In [24]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(44, 88)
        self.fc2 = nn.Linear(88, 176)
        self.fc3 = nn.Linear(176, 352)
        self.fc4 = nn.Linear(352, 176)
        self.fc5 = nn.Linear(176, 88)
        self.fc6 = nn.Linear(88, 44)
        self.fc7 = nn.Linear(44, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = torch.sigmoid(self.fc7(x))
        return x

In [25]:
model_info = {
    "name": "ANN",
    "arch": "44-88-176-352-176-88-44-10",
    "criterion": "BCELoss",
    "lr":0.001,
    "epoch": 300,
    "batch_size":128,
    "optimizer":"Adam",
    "activation":"Sigmoid",
    "features": "Selected"
}

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 300
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 9.427347183227539
  Train Song Accuracy: 0.10375
  Train Accuracy: 0.09006755066299725
  Validation Loss: 9.305908203125
  Validation Song Accuracy: 0.07
  Validation Accuracy: 0.07064128256513026
Epoch 2:
  Train Loss: 9.76314926147461
  Train Song Accuracy: 0.10875
  Train Accuracy: 0.1057042782086565
  Validation Loss: 9.794241905212402
  Validation Song Accuracy: 0.08
  Validation Accuracy: 0.09869739478957916
Epoch 3:
  Train Loss: 17.801362991333008
  Train Song Accuracy: 0.1
  Train Accuracy: 0.09994996247185389
  Validation Loss: 17.545242309570312
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 4:
  Train Loss: 17.725610733032227
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10020015011258444
  Validation Loss: 17.481740951538086
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 5:
  Train Loss: 9.793306350708008
  Train Song Accuracy: 0.1375
  Train Accuracy: 0.11796347260445333
  Validati

In [27]:
save_model(model, model_info, "ANN-selected", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)

Converting 'ANN-selected' to lowercase: 'ann-selected'.


saved
logged


## Run 4
- Try softmax with CrossEntrophy instead (Multi class classification)

In [28]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(44, 88)
        self.fc2 = nn.Linear(88, 88)
        self.fc3 = nn.Linear(88, 44)
        self.fc4 = nn.Linear(44, 10)  # Adjust output size based on the number of classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.softmax(self.fc4(x),1)
        return x

In [36]:
model_info = {
    "name": "ANN",
    "arch": "44-88-88-88-88-88-88-44-10",
    "criterion": "CELoss",
    "lr":0.0011,
    "epoch": 300,
    "batch_size":128,
    "optimizer":"SGD",
    "activation":"Softmax",
    "features": "Selected"
}

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.SGD(model.parameters(), lr=0.0011)
num_epochs = 300
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 2.303931713104248
  Train Song Accuracy: 0.14375
  Train Accuracy: 0.14323242431823868
  Validation Loss: 2.318094253540039
  Validation Song Accuracy: 0.115
  Validation Accuracy: 0.125751503006012
Epoch 2:
  Train Loss: 2.2776975631713867
  Train Song Accuracy: 0.18625
  Train Accuracy: 0.17087815861896422
  Validation Loss: 2.3067967891693115
  Validation Song Accuracy: 0.11
  Validation Accuracy: 0.12675350701402804
Epoch 3:
  Train Loss: 2.2760586738586426
  Train Song Accuracy: 0.1975
  Train Accuracy: 0.17950963222416813
  Validation Loss: 2.30141282081604
  Validation Song Accuracy: 0.135
  Validation Accuracy: 0.14529058116232466
Epoch 4:
  Train Loss: 2.282576322555542
  Train Song Accuracy: 0.19125
  Train Accuracy: 0.17638228671503628
  Validation Loss: 2.302685499191284
  Validation Song Accuracy: 0.13
  Validation Accuracy: 0.15080160320641284
Epoch 5:
  Train Loss: 2.280958414077759
  Train Song Accuracy: 0.185
  Train Accuracy: 0.1647485614210658


In [31]:
save_model(model, model_info, "ANN-Softmax", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)

Converting 'ANN-Softmax' to lowercase: 'ann-softmax'.


saved
logged


## Run 5
- Deal with overfitting
- Model architecture from Run 2, because the training accuracy is the highest in that version

In [32]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(44, 88)
        self.fc2 = nn.Linear(88, 88)
        self.fc3 = nn.Linear(88, 176)
        self.fc4 = nn.Linear(176, 176)
        self.fc5 = nn.Linear(176, 88)
        self.fc6 = nn.Linear(88, 44)
        self.fc7 = nn.Linear(44, 10)
        self.dropout = nn.Dropout(0.1)
        

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout(x)
        x = torch.relu(self.fc5(x))
        x = self.dropout(x)
        x = torch.relu(self.fc6(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc7(x))
        return x

In [44]:
model_info = {
    "name": "ANN",
    "arch": "44-88-88-88-176-176-88-44-10",
    "criterion": "BCELoss",
    "lr":0.0011,
    "epoch": 500,
    "batch_size":128,
    "optimizer":"Adam",
    "activation":"Sigmoid",
    "features": "Selected",
    "dropout": 0.1
}

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.0011)
num_epochs = 500
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 0.48201361298561096
  Train Song Accuracy: 0.1075
  Train Accuracy: 0.10595446584938704
  Validation Loss: 0.4838806092739105
  Validation Song Accuracy: 0.095
  Validation Accuracy: 0.11022044088176353
Epoch 2:
  Train Loss: 0.4213312864303589
  Train Song Accuracy: 0.1075
  Train Accuracy: 0.11371028271203402
  Validation Loss: 0.4251749515533447
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10070140280561123
Epoch 3:
  Train Loss: 0.3889123797416687
  Train Song Accuracy: 0.12875
  Train Accuracy: 0.14473355016262196
  Validation Loss: 0.39762023091316223
  Validation Song Accuracy: 0.12
  Validation Accuracy: 0.1342685370741483
Epoch 4:
  Train Loss: 0.3654334843158722
  Train Song Accuracy: 0.185
  Train Accuracy: 0.1757568176132099
  Validation Loss: 0.37172210216522217
  Validation Song Accuracy: 0.15
  Validation Accuracy: 0.15581162324649298
Epoch 5:
  Train Loss: 0.361697793006897
  Train Song Accuracy: 0.21875
  Train Accuracy: 0.20065048786

In [39]:
save_model(model, model_info, "ANN-selected", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)

Converting 'ANN-selected' to lowercase: 'ann-selected'.


saved
logged


## Run 6
- Deal with overfitting

In [40]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(44, 88)
        self.fc2 = nn.Linear(88, 88)
        self.fc3 = nn.Linear(88, 176)
        self.fc4 = nn.Linear(176, 176)
        self.fc5 = nn.Linear(176, 88)
        self.fc6 = nn.Linear(88, 44)
        self.fc7 = nn.Linear(44, 10)
        self.dropout = nn.Dropout(0.05)
        

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout(x)
        x = torch.relu(self.fc5(x))
        x = self.dropout(x)
        x = torch.relu(self.fc6(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc7(x))
        return x

In [78]:
model_info = {
    "name": "ANN",
    "arch": "44-88-88-88-176-176-88-44-10",
    "criterion": "BCELoss",
    "lr":0.0011,
    "epoch": 500,
    "batch_size":128,
    "optimizer":"Adam",
    "activation":"Sigmoid",
    "features": "Selected",
    "dropout": 0.05
}

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.0011)
num_epochs = 500
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 0.42054158449172974
  Train Song Accuracy: 0.1875
  Train Accuracy: 0.16599949962471855
  Validation Loss: 0.42871415615081787
  Validation Song Accuracy: 0.165
  Validation Accuracy: 0.12474949899799599
Epoch 2:
  Train Loss: 0.3824392557144165
  Train Song Accuracy: 0.18
  Train Accuracy: 0.16449837378033524
  Validation Loss: 0.3930880129337311
  Validation Song Accuracy: 0.175
  Validation Accuracy: 0.16182364729458917
Epoch 3:
  Train Loss: 0.3689064681529999
  Train Song Accuracy: 0.235
  Train Accuracy: 0.20878158618964224
  Validation Loss: 0.378501832485199
  Validation Song Accuracy: 0.2
  Validation Accuracy: 0.18887775551102204
Epoch 4:
  Train Loss: 0.36368003487586975
  Train Song Accuracy: 0.27
  Train Accuracy: 0.22604453340005004
  Validation Loss: 0.3732755482196808
  Validation Song Accuracy: 0.22
  Validation Accuracy: 0.19288577154308617
Epoch 5:
  Train Loss: 0.344417929649353
  Train Song Accuracy: 0.245
  Train Accuracy: 0.2181636227170378

In [46]:
save_model(model, model_info, "ANN-selected", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)

Converting 'ANN-selected' to lowercase: 'ann-selected'.


saved
logged


## Run 7
- Deal with overfitting

In [47]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(44, 88)
        self.fc2 = nn.Linear(88, 88)
        self.fc3 = nn.Linear(88, 176)
        self.fc4 = nn.Linear(176, 176)
        self.fc5 = nn.Linear(176, 88)
        self.fc6 = nn.Linear(88, 44)
        self.fc7 = nn.Linear(44, 10)
        self.dropout = nn.Dropout(0.075)
        

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout(x)
        x = torch.relu(self.fc5(x))
        x = self.dropout(x)
        x = torch.relu(self.fc6(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc7(x))
        return x

In [48]:
model_info = {
    "name": "ANN",
    "arch": "44-88-88-88-176-176-88-44-10",
    "criterion": "BCELoss",
    "lr":0.0011,
    "epoch": 500,
    "batch_size":128,
    "optimizer":"Adam",
    "activation":"Sigmoid",
    "features": "Selected",
    "dropout": 0.075
}

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.0011)
num_epochs = 500
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 0.4171254336833954
  Train Song Accuracy: 0.1375
  Train Accuracy: 0.13422566925193896
  Validation Loss: 0.42305684089660645
  Validation Song Accuracy: 0.14
  Validation Accuracy: 0.14579158316633267
Epoch 2:
  Train Loss: 0.38840773701667786
  Train Song Accuracy: 0.20375
  Train Accuracy: 0.18163622717037778
  Validation Loss: 0.3932579457759857
  Validation Song Accuracy: 0.18
  Validation Accuracy: 0.16382765531062124
Epoch 3:
  Train Loss: 0.37216052412986755
  Train Song Accuracy: 0.195
  Train Accuracy: 0.18651488616462347
  Validation Loss: 0.3794841468334198
  Validation Song Accuracy: 0.2
  Validation Accuracy: 0.19839679358717435
Epoch 4:
  Train Loss: 0.365352988243103
  Train Song Accuracy: 0.20625
  Train Accuracy: 0.19427070302727045
  Validation Loss: 0.37402400374412537
  Validation Song Accuracy: 0.22
  Validation Accuracy: 0.17985971943887777
Epoch 5:
  Train Loss: 0.35359370708465576
  Train Song Accuracy: 0.23125
  Train Accuracy: 0.2141606

In [50]:
save_model(model, model_info, "ANN-selected", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)

Converting 'ANN-selected' to lowercase: 'ann-selected'.


saved
logged


## Export model
- Doesn't have enough time to build/deploy the model service API
- Not included in the requirements anyway
- Export models to .pth files instead

In [80]:
load_model = bentoml.pytorch.load_model('ann-selected:dsxu2paj76qhhpap', device_id="cuda")
torch.save(load_model.state_dict(), "model_state\selected_7.pth")

In [26]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(44, 88)
        self.fc2 = nn.Linear(88, 88)
        self.fc3 = nn.Linear(88, 44)
        self.fc4 = nn.Linear(44, 10)
        

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

In [19]:
model_info = {
    "name": "ANN",
    "arch": "44-88-176-352-176-88-44-10",
    "criterion": "BCELoss",
    "lr":0.0011,
    "epoch": 300,
    "batch_size":128,
    "optimizer":"Adam",
    "activation":"Sigmoid",
    "features": "Selected",
    "dropout": 0.075
}

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.0011)
num_epochs = 100
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 26.00200080871582
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10007505629221916
  Validation Loss: 25.989606857299805
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 2:
  Train Loss: 26.00200080871582
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10007505629221916
  Validation Loss: 25.989593505859375
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 3:
  Train Loss: 26.00200080871582
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10007505629221916
  Validation Loss: 25.989593505859375
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 4:
  Train Loss: 26.00200080871582
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10007505629221916
  Validation Loss: 25.989593505859375
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 5:
  Train Loss: 26.00200080871582
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10007505629221916
  Validation Loss:

In [ ]:
save_model(model, model_info, "ANN-selected", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)